In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
import logging
import shutil
import os

logging.basicConfig(level=logging.INFO)

train_path = "/kaggle/input/arabic-dataset/Unbalanced/Unbalanced/MAQA_Train.xlsx"
test_path = "/kaggle/input/arabic-dataset/Unbalanced/Unbalanced/MAQA_Test.xlsx"

valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]

train_df = pd.read_excel(train_path)
train_df = train_df[['q_body', 'category']]
train_df = train_df[train_df["category"].isin(valid_categories)]
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

test_df = pd.read_excel(test_path)
test_df = test_df[['q_body', 'category']]
test_df = test_df[test_df["category"].isin(valid_categories)]
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n🔹 Training Dataset Class Distribution:")
print(train_df["category"].value_counts())

print("\n🔹 Test Dataset Class Distribution:")
print(test_df["category"].value_counts())

category_mapping = {cat: i for i, cat in enumerate(valid_categories)}
train_df['label'] = train_df['category'].map(category_mapping)
test_df['label'] = test_df['category'].map(category_mapping)


🔹 Training Dataset Class Distribution:
category
امراض نسائية                       56128
انف اذن وحنجرة                     15646
امراض العضلات والعظام و المفاصل    14848
امراض العيون                       14639
امراض القلب و الشرايين             12756
امراض الجهاز الهضمي                12708
الامراض الجنسية                     8874
طب الاسنان                          8805
جراحة تجميل                         7875
امراض الدم                          6861
Name: count, dtype: int64

🔹 Test Dataset Class Distribution:
category
امراض نسائية                       14032
انف اذن وحنجرة                      3912
امراض العضلات والعظام و المفاصل     3712
امراض العيون                        3660
امراض القلب و الشرايين              3190
امراض الجهاز الهضمي                 3177
الامراض الجنسية                     2219
طب الاسنان                          2202
جراحة تجميل                         1969
امراض الدم                          1716
Name: count, dtype: int64


In [ ]:
model_name = "aubmindlab/bert-base-arabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["q_body"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df[['q_body', 'label']])
test_dataset = Dataset.from_pandas(test_df[['q_body', 'label']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

base_model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/717k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/159140 [00:00<?, ? examples/s]

Map:   0%|          | 0/39789 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [4]:
class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}
model = CustomModel(base_model, num_labels=len(valid_categories))


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  
    save_strategy="no",           
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=3e-5,
    load_best_model_at_end=False,  
    disable_tqdm=False,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
print("\n🔹 Starting Training...")
trainer.train()

test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)



<ipython-input-6-75d86d42feae>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🔹 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.333700,0.335430,0.897384,0.897438,0.897384,0.896437
2,0.239800,0.297772,0.906482,0.906228,0.906482,0.905240
3,0.213200,0.303569,0.908191,0.908102,0.908191,0.907430
4,0.122400,0.333020,0.908492,0.909718,0.908492,0.908638
5,0.084200,0.359928,0.911357,0.911735,0.911357,0.910965
6,0.073200,0.404680,0.911835,0.911746,0.911835,0.911097
7,0.046800,0.444638,0.911207,0.910606,0.911207,0.910542
8,0.033300,0.478768,0.913820,0.912947,0.913820,0.912933
9,0.024100,0.514985,0.911634,0.911182,0.911634,0.911112
10,0.021500,0.521269,0.913971,0.913197,0.913971,0.913262



🔹 Test Metrics: {'eval_loss': 0.5212693214416504, 'eval_accuracy': 0.9139711980698183, 'eval_precision': 0.9131974057295847, 'eval_recall': 0.9139711980698183, 'eval_f1': 0.9132615790320987, 'eval_runtime': 144.5375, 'eval_samples_per_second': 275.285, 'eval_steps_per_second': 2.871, 'epoch': 10.0}


In [ ]:
def save_complete_model(model, tokenizer, category_mapping, save_path="AraBert"):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    classifier_state = {
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }
    torch.save(classifier_state, f"{save_path}/classifier_state.pt")
    
    with open(f"{save_path}/category_mapping.pkl", "wb") as f:
        pickle.dump(category_mapping, f)
# Save the model
save_complete_model(trainer.model, tokenizer, category_mapping, "AraBert")

shutil.make_archive("AraBert", 'zip', "AraBert")
print("\n✅ Model Saved as 'AraBert' & Zipped for Download!")

from IPython.display import FileLink
FileLink(r'AraBert.zip')


✅ Model Saved as 'AraBert' & Zipped for Download!


/kaggle/working/AraBert.zip